In [ ]:
import numpy as np

from plotly.subplots import make_subplots
import plotly.graph_objects as go
import plotly.express as px

from sklearn.ensemble import IsolationForest
from sklearn.svm import OneClassSVM

from ocml.datasets import load_toy_2d, default_process


def get_sk_contour(model, domain, resolution):
  """Return grid with level sets from model.
  
  Args:
      model: the classifier to plot
      domain: domain on which to plot the classifier level sets
      resolution: number of pixels in discretization
      
  Returns:
      grid with predictions, x and y coordinnaates of the grid. 
  """
  xmin, xmax = domain[0], domain[1]
  ymin, ymax = domain[0], domain[1]
  x_coords = np.linspace(xmin, xmax, resolution)
  y_coords = np.linspace(ymin, ymax, resolution)
  x_grid, y_grid = np.meshgrid(x_coords, y_coords)
  grid = np.stack([x_grid.flatten(), y_grid.flatten()], axis=-1)
  z_grid = model.decision_function(grid)
  z_grid = z_grid.reshape(x_grid.shape)
  return z_grid, x_coords, y_coords, grid

def plot_sk_contour(model, X, y,
                    fig, col,
                    domain=None):  # domain on which to plot the data.
  if domain is None:
    domain = config.domain
  z_grid, x_coords, y_coords, grid = get_sk_contour(model, domain, resolution=300)    
  fig.add_trace(go.Contour(z=z_grid, x=x_coords, y=y_coords, ncontours=20), row=1, col=col)
  fig.add_trace(go.Scatter(x=X[:,0], y=X[:,1], mode='markers', marker=dict(size=1.5, color='black')), row=1, col=col)
  return fig

In [ ]:
def get_sk_model(sk_model_name):
  outliers_fraction = 0.05
  if sk_model_name == 'if':
    sk_model = IsolationForest(contamination=outliers_fraction)
  elif sk_model_name == 'ocsvm':
    sk_model = OneClassSVM(nu=outliers_fraction, kernel="rbf", gamma='scale')
  return sk_model

In [ ]:
def train_plot_sk_model(fig, col, ds_name, sk_model_name, domain):
  X, y = load_toy_2d(config.dataset_name, config.num_pts, config.noise)
  X, y = default_process(X, y)
  sk_model = get_sk_model(sk_model_name)
  sk_model.fit(X, y)
  plot_sk_contour(sk_model, X, y, fig, col, domain=domain)

In [ ]:
def train_plot_sk_model(fig, col, ds_name, sk_model_name, domain):
  X, y = load_toy_2d(config.dataset_name, config.num_pts, config.noise)
  X, y = default_process(X, y)
  sk_model = get_sk_model(sk_model_name)
  sk_model.fit(X, y)
  plot_sk_contour(sk_model, X, y, fig, col, domain=domain)

In [ ]:
num_cols = 1
fig = make_subplots(rows=1, cols=num_cols, shared_yaxes=True)
sk_model_name = "ocsvm"  # or "if"
train_plot_sk_model(fig, 1, config.dataset_name, sk_model_name, domain=[-3, 3])
width = num_cols * 500
fig.update_layout(autosize=False, width=width, height=500, showlegend=False, barmode='overlay')
fig.update_yaxes(
    scaleanchor = "x",
    scaleratio = 1,
    row=1,
    col=1,
)